# use ollama and vectorsotre to build RAG system

procedures:
1. install ollma, try putting models in /kaggle/working so that it will save to next run.
2. install other packages, follow the code in [langchain guide](https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_adaptive_rag_local/). will need to change langchain_community.document_loaders from webpage to pdf. can test pdf first, then try md.
3. open a port to public by gradio.
4. finally try to find a way to make the response contain graph

If 1-3 is finished, it will be equivalent to google's notenookLM. step 4 is beyond that.

In [1]:
!pwd

/kaggle/working


## install ollama

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

curl: /opt/conda/lib/libcurl.so.4: no version information available (required by curl)
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
curl: /opt/conda/lib/libcurl.so.4: no version information available (required by curl)
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


## install python packages

start from this step if you already have ollama

In [ ]:
%%capture --no-stderr
%pip install ollama pypdf Markdown unstructured
%pip install --quiet -U langchain langchain_community tiktoken langchain-nomic "nomic[local]" langchain-ollama scikit-learn langgraph tavily-python bs4
%pip install langchain-nomic

In [2]:
import ollama
ollama.list()

ListResponse(models=[Model(model='qwen2.5-coder:14b', modified_at=datetime.datetime(2024, 11, 18, 18, 41, 51, 756602, tzinfo=TzInfo(-05:00)), digest='3028237cc8c52fea4e77185d72cc997b2e90392791f7c82fe1c71995d56e642d', size=8988124256, details=ModelDetails(parent_model='', format='gguf', family='qwen2', families=['qwen2'], parameter_size='14.8B', quantization_level='Q4_K_M')), Model(model='llama3.2-vision:latest', modified_at=datetime.datetime(2024, 11, 18, 17, 41, 39, 984314, tzinfo=TzInfo(-05:00)), digest='38107a0cd11910a31c300fcfd1e9a107b2928e56ebabd14598702170b004773e', size=7901829417, details=ModelDetails(parent_model='', format='gguf', family='mllama', families=['mllama', 'mllama'], parameter_size='9.8B', quantization_level='Q4_K_M'))])

## install llama3.2

In [ ]:
# no need to install if you already have it
#import subprocess
#process = subprocess.Popen("ollama serve", shell=True) #runs on a different thread
#!ollama pull llama3.2:3b-instruct-fp16

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIE2L1pBL+tujLha+6BSXYS7JlR9se5Gmgmm+GzNTWgyo



2024/12/12 16:37:30 routes.go:1195: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://*] OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES: http_proxy: https_proxy: no_proxy:]"
time=2024-12-12T16:37:30.638Z level=INF

[GIN] 2024/12/12 - 16:37:31 | 200 |      68.319µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling e2f46f5b501c...   0% ▕                ▏    0 B/6.4 GB                  

time=2024-12-12T16:37:32.665Z level=INFO source=download.go:175 msg="downloading e2f46f5b501c in 16 402 MB part(s)"


pulling manifest 
pulling e2f46f5b501c...   0% ▕                ▏    0 B/6.4 GB                  pulling manifest 
pulling e2f46f5b501c...   0% ▕                ▏    0 B/6.4 GB                  pulling manifest 
pulling e2f46f5b501c...   0% ▕                ▏    0 B/6.4 GB                  pulling manifest 
pulling e2f46f5b501c...   0% ▕                ▏ 3.5 MB/6.4 GB                  pulling manifest 
pulling e2f46f5b501c...   1% ▕                ▏  51 MB/6.4 GB                  pulling manifest 
pulling e2f46f5b501c...   1% ▕                ▏  87 MB/6.4 GB                  pulling manifest 
pulling e2f46f5b501c...   3% ▕                ▏ 168 MB/6.4 GB                  pulling manifest 
pulling e2f46f5b501c...   4% ▕                ▏ 246 MB/6.4 GB                  pulling manifest 
pulling e2f46f5b501c...   4% ▕                ▏ 280 MB/6.4 GB                  pulling manifest 
pulling e2f46f5b501c...   6% ▕                ▏ 358 MB/6.4 GB  358 MB/s     16spulling manifest 
pulling e2f4

...

pulling manifest 
pulling e2f46f5b501c... 100% ▕████████████████▏ 6.4 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9...   0% ▕                ▏    0 B/6.0 KB                  pulling manifest 
pulling e2f46f5b501c... 100% ▕████████████████▏ 6.4 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         pulling manifest 
pulling e2f46f5b501c... 100% ▕████████████████▏ 6.4 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB              

time=2024-12-12T16:37:52.951Z level=INFO source=download.go:175 msg="downloading 56bb8bd477a5 in 1 96 B part(s)"


pulling manifest 
pulling e2f46f5b501c... 100% ▕████████████████▏ 6.4 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5...   0% ▕                ▏    0 B/  96 B                  pulling manifest 
pulling e2f46f5b501c... 100% ▕████████████████▏ 6.4 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         pulling manifest 
pulling e2f46f5b501c... 100% ▕████████████████▏ 6.4 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB              

time=2024-12-12T16:37:54.272Z level=INFO source=download.go:175 msg="downloading 1bc315994ceb in 1 558 B part(s)"


pulling manifest 
pulling e2f46f5b501c... 100% ▕████████████████▏ 6.4 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 1bc315994ceb...   0% ▕                ▏    0 B/ 558 B                  pulling manifest 
pulling e2f46f5b501c... 100% ▕████████████████▏ 6.4 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 1bc315994ceb... 100% ▕████████████████▏  558 B                         pullin

## initial test

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from IPython.display import display, Markdown
import ollama

response = ollama.chat(model='llama3.2:3b-instruct-fp16', messages=[
  {
    'role': 'user',
    'content': 'summarize properties of chalcogenide perovskites',
  },
])
Markdown(response['message']['content'])

Chalcogenide perovskites are a class of semiconducting materials that have gained significant attention in recent years due to their unique properties and potential applications. Here are some key properties of chalcogenide perovskites:

**Crystal Structure:**

* Chalcogenide perovskites have a general formula of ABx, where A is a metal halide (e.g., MA+ or M2+), B is a tin-based cation (Sn2+), and x is the counterion (e.g., Cl-, Br-).
* They exhibit a perovskite crystal structure with a distorted octahedral arrangement of cations.

**Optical Properties:**

* High transparency in the visible and near-infrared spectrum, making them suitable for optoelectronic applications.
* Strong absorption in the UV/visible range, which can be used to enhance light harvesting or create selective filters.
* High refractive index (n) and low dielectric constant (ε), leading to high sensitivity and fast response times.

**Electrical Properties:**

* High carrier mobility (μ) due to the presence of Sn2+, allowing for efficient charge transport.
* High conductivity in the forward direction, but can exhibit high resistivity in reverse directions.
* Low trap state density, which helps to minimize non-radiative recombination and increase efficiency.

**Thermal Properties:**

* High thermal stability up to 300°C or more, making them suitable for use in high-temperature applications.
* High thermal conductivity, allowing for efficient heat dissipation.

**Stability and Durability:**

* Generally less stable than organic halide perovskites, but can exhibit improved stability with certain dopants or passivation techniques.
* Susceptible to degradation upon exposure to moisture, light, or temperature fluctuations.

**Potential Applications:**

* Photovoltaics (PV) devices, such as solar cells and photodetectors.
* Display technologies, including LEDs and OLEDs.
* Sensing applications, like gas sensors and chemical sensors.
* Therapeutic applications, such as tissue engineering scaffolds.

Overall, chalcogenide perovskites offer a promising platform for the development of high-performance optoelectronic devices with improved stability and durability.

# try RAG

## LLM

In [9]:

from langchain_ollama import ChatOllama

local_llm = "llama3.2:3b-instruct-fp16"
llm = ChatOllama(model=local_llm, temperature=0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0, format="json",num_ctx=4096)

## vector store

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_nomic.embeddings import NomicEmbeddings
from langchain.document_loaders import PyPDFLoader

pdf_path = [
    "pdf_files/1.pdf",
    "pdf_files/2.pdf",
    "pdf_files/3.pdf",
    "pdf_files/4.pdf"
]

# Load documents
docs = [PyPDFLoader(pdf).load() for pdf in pdf_path]
docs_list = [item for sublist in docs for item in sublist]

# Split documents
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=200
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorDB
vectorstore = SKLearnVectorStore.from_documents(
    documents=doc_splits,
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
)

# Create retriever
retriever = vectorstore.as_retriever(k=4) # k: Amount of documents to return (Default: 4)

Downloading: 100%|██████████| 274M/274M [00:07<00:00, 34.6MiB/s] 
Verifying: 100%|██████████| 274M/274M [00:00<00:00, 813MiB/s] 


## Components

In [9]:
### Router
import json
from langchain_core.messages import HumanMessage, SystemMessage

# Prompt
router_instructions = """You are an expert at routing a user question to a vectorstore.

The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.

Use the vectorstore for questions on these topics.

Return JSON with single key, datasource, that is 'vectorstore'."""



## Retrieval Grader

## Generate

In [10]:
### Generate
import warnings
warnings.filterwarnings("ignore")
from langchain_core.messages import HumanMessage
# Prompt
question = "summarize properties of chalcogenide perovskites"

rag_prompt = """You are an assistant for question-answering tasks. 

Here is the context to use to answer the question:

{context} 

Think carefully about the above context. 

Now, review the user question:

{question}

Provide an answer to this questions using only the above context. 


Answer:"""


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Test
docs = retriever.invoke(question)
docs_txt = format_docs(docs)
rag_prompt_formatted = rag_prompt.format(context=docs_txt, question=question)
generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])
print(generation.content)

Chalcogenide perovskites are a novel class of semiconductor materials that have emerged as a promising material for optoelectronic applications. They exhibit properties intermediate between covalency and ionicity, offering opportunities for discovering semiconductors with unique electronic and optical properties.

Some notable properties of chalcogenide perovskites include:

* Direct bandgap semiconductors
* Strong light absorption
* Good carrier mobility
* Ionic bonding, which provides a strong electron correlation but reduced dielectric screening compared to conventional oxides or halides

Additionally, these materials have been found to exhibit giant optical anisotropy in certain quasi-one-dimensional crystals. They also show promise for photovoltaic applications and have been the subject of extensive research in recent years.

It's worth noting that chalcogenide perovskites are more ionic than conventional semiconductors but less so than oxides or halides, making them an interestin

# RAG_pdf class
make a class to simplify workflow

In [1]:
import glob
from IPython.display import display, Markdown
import ollama
from langchain_ollama import ChatOllama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_nomic.embeddings import NomicEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain_core.messages import HumanMessage
from langchain_ollama import OllamaEmbeddings
local_llm = "llama3.2:3b-instruct-fp16"
class RAG_by_pdf:
    """
    Define the class that uses PDF as input to implement RAG.
    """

    def __init__(self, question: str, pdf_folder: str, model: str= local_llm,num_retrived_docs: int = 2):
        """
        Initialize the RAG_by_pdf class.

        Args:
        - question (str): The question to be answered.
        - pdf_folder (str): The folder containing the PDF files.
        - num_retrived_docs (int, optional): The number of documents to retrieve. Defaults to 4.
        """
        self.pdf_path = glob.glob(pdf_folder + "/*.pdf")
        self.num_retrived_docs = num_retrived_docs
        self.question = question
        self.model=model
        self.rag_prompt = """You are an assistant for question-answering tasks. 
        Here is the context to use to answer the question:
        {context} 
        Think carefully about the above context. 
        Now, review the user question:
        {question}
        Provide an answer to this questions using only the above context. 
        Answer:"""

    def retriever(self) -> SKLearnVectorStore:
        """
        Create a retriever based on the PDF documents.

        Returns:
        - SKLearnVectorStore: The retriever.
        """
        try:
            # Load documents
            docs = [PyPDFLoader(pdf).load() for pdf in self.pdf_path]
            docs_list = [item for sublist in docs for item in sublist]

            # Split documents
            text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
                chunk_size=1000, chunk_overlap=200
            )
            doc_splits = text_splitter.split_documents(docs_list)

            # Add to vectorDB
            vectorstore = SKLearnVectorStore.from_documents(
                documents=doc_splits,
                #embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
                embedding=OllamaEmbeddings(model=self.model)
            )

            return vectorstore
        except Exception as e:
            print(f"Error creating retriever: {e}")
            return None

    def format_docs(self, docs):
        """
        Format the retrieved documents.

        Args:
        - docs: The retrieved documents.

        Returns:
        - str: The formatted documents.
        """
        return "\n\n".join(doc.page_content for doc in docs)

    def return_context(self) -> str:
        """
        Return the context based on the question.

        Returns:
        - str: The context.
        """
        vectorstore = self.retriever()
        if vectorstore is not None:
            # Create retriever
            retriever = vectorstore.as_retriever(k=self.num_retrived_docs)
            docs = retriever.invoke(self.question)
            docs_txt = self.format_docs(docs)
            rag_prompt_formatted = self.rag_prompt.format(context=docs_txt, question=self.question)
            return rag_prompt_formatted
        else:
            return ""

In [32]:
question = "summarize properties of chalcogenide perovskites"
pdf_folder=r"C:\Users\haoze\Documents\python_scripts\haolei\RAG\pdf_files"
rag=RAG_by_pdf(question,pdf_folder)
content=rag.return_context()

In [33]:
local_llm = "llama3.2:3b-instruct-fp16"
llm = ChatOllama(model=local_llm, temperature=0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0, format="json",num_ctx=4096)

In [34]:
generation = llm.invoke([HumanMessage(content=content)])
print(generation.content)

Based on the provided context, here is a summary of the properties of chalcogenide perovskites:

**Definition**: Chalcogenide perovskites are an emerging class of ionic semiconductors.

**Properties mentioned in the context**:

* They can be used for photovoltaic applications.
* They have giant optical anisotropy, which means they exhibit significant differences in their optical properties depending on the direction of light incidence.
* They can be alloyed and defects controlled to optimize their performance for photovoltaic applications.
* They are stable under high pressures.
* They have a wide range of potential applications, including solar energy conversion, thermoelectric devices, and excited state dynamics.

**Other properties implied by the context**:

* They are low-cost materials.
* They can exhibit unique electronic structures and optical properties due to their ionic nature.
* They may be used for other applications beyond photovoltaics, such as spintronics and magnetic ma

# RAG_markdown class

by using ollama embedings, retrieveing time consumed decreased from 1min40s to 12s

In [2]:
import glob
from IPython.display import display, Markdown
import ollama
from langchain_ollama import ChatOllama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_nomic.embeddings import NomicEmbeddings
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.messages import HumanMessage,SystemMessage
from langchain_ollama import OllamaEmbeddings

local_llm = "llama3.2:3b-instruct-fp16"

class RAG_by_markdown:
    """
    Define the class that uses Markdown as input to implement RAG.
    """

    def __init__(self, question: str, markdown_folder: str, model: str= local_llm, num_retrived_docs: int = 2):
        """
        Initialize the RAG_by_markdown class.

        Args:
        - question (str): The question to be answered.
        - markdown_folder (str): The folder containing the Markdown files.
        - model(str): the model used for retriever
        - num_retrived_docs (int, optional): The number of documents to retrieve. Defaults to 4.
        """
        self.markdown_path = glob.glob(markdown_folder + "/*.md")
        self.num_retrived_docs = num_retrived_docs
        self.model=model
        self.question = question
        self.rag_prompt =  """You are an assistant for question-answering tasks. 

        Here is the context to use to answer the question:

        {context} 

        Think carefully about the above context. 

        Now, review the user question:

        {question}

        Provide an answer to this questions using only the above context. 

        Use three sentences maximum and keep the answer concise.

        Answer:"""


    def retriever(self) -> SKLearnVectorStore:
        """
        Create a retriever based on the Markdown documents.

        Returns:
        - SKLearnVectorStore: The retriever.
        """
        try:
            # Load documents
            docs = [UnstructuredMarkdownLoader(md).load() for md in self.markdown_path]
            docs_list = [item for sublist in docs for item in sublist]

            # Split documents
            text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
                chunk_size=1000, chunk_overlap=200
            )
            doc_splits = text_splitter.split_documents(docs_list)

            # Add to vectorDB
            vectorstore = SKLearnVectorStore.from_documents(
                documents=doc_splits,
                #embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
                embedding=OllamaEmbeddings(model=self.model)
            )

            return vectorstore
        except Exception as e:
            print(f"Error creating retriever: {e}")
            return None

    def format_docs(self, docs):
        """
        Format the retrieved documents.

        Args:
        - docs: The retrieved documents.

        Returns:
        - str: The formatted documents.
        """
        return "\n\n".join(doc.page_content for doc in docs)

    def return_context(self) -> str:
        """
        Return the context based on the question.

        Returns:
        - str: The context.
        """
        vectorstore = self.retriever()
        if vectorstore is not None:
            # Create retriever
            retriever = vectorstore.as_retriever(k=self.num_retrived_docs)
            docs = retriever.invoke(self.question)
            docs_txt = self.format_docs(docs)
            rag_prompt_formatted = self.rag_prompt.format(context=docs_txt, question=self.question)
            return rag_prompt_formatted
        else:
            return ""


router_instructions = """You are an expert at routing a user question to a vectorstore.

The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.

Use the vectorstore for questions on these topics.

Return JSON with single key, datasource depending on the question."""

In [29]:
# Example usage
md_folder= r"C:\Users\haoze\Documents\python_scripts\haolei\RAG\pdf_files"
question = "what is advantages of chalcogenide perovskites"
rag = RAG_by_markdown(question, md_folder)
content = rag.return_context()
print(content)

You are an assistant for question-answering tasks. 

        Here is the context to use to answer the question:

        In this paper we report the synthesis of $\mathrm{BaZrS}{3}$ thin films at tem­ peratures as low as $500\,^{\circ}\mathrm{C}$ , which is comparable to the processing temperature of copper indium gallium selenide (CIGS) solar cells [31, 32]. This is achieved by changing the chemical reaction pathway from sulfurization of oxide perovskites to crystallization of pulsed laser deposited amorphous $\mathrm{\ttBaZrS{x}}$ films. The films processed at low temper­ atures show stoichiometric single-phase orthorhombic perovskite structure with smooth surfaces. Elimination of excess sulfur vacancies leads to reduced carrier concentration to $\sim!4\times10^{10}\ensuremath{~\mathrm{\mm}^{-3}}$ . As a demonstration for the potential use of the $\mathrm{BaZrS}{3}$ thin films towards various electronic and optoelectronic applications, $\mathrm{BaZrS}{3}$ thin-film photodetectors and

In [36]:
local_llm = "llama3.2:3b-instruct-fp16"
llm = ChatOllama(model=local_llm, temperature=0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0, format="json",num_ctx=4096)
#llm.reset()
generation = llm.invoke([SystemMessage(content=router_instructions)]+[HumanMessage(content=content)])
print(generation.content)

Based on the provided context, here is a summary of the properties of chalcogenide perovskites:

**Definition**: Chalcogenide perovskites are an emerging class of ionic semiconductors.

**Properties mentioned in the context**:

* They can be used for photovoltaic applications.
* They have giant optical anisotropy, which means they exhibit significant differences in their optical properties depending on the direction of light incidence.
* They can be alloyed and defects controlled to optimize their performance for photovoltaic applications.
* They are stable under high pressures.
* They have a wide range of potential applications, including solar energy conversion, thermoelectric devices, and excited state dynamics.

**Other properties implied by the context**:

* They are low-cost materials.
* They can exhibit unique electronic structures and optical properties due to their ionic nature.
* They may be used for other applications beyond photovoltaics, such as spintronics and magnetic ma

# gradio interface
features:

1. choose models for embedings and generation
2. choose pdf or markdown rag
3. input questions
4. get answer

In [7]:
import gradio as gr
md_folder= r"C:\Users\haoze\Documents\python_scripts\haolei\RAG\pdf_files"
pdf_folder=r"C:\Users\haoze\Documents\python_scripts\haolei\RAG\pdf_files"
import glob
from IPython.display import display, Markdown
from langchain_ollama import ChatOllama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_nomic.embeddings import NomicEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain_core.messages import HumanMessage,SystemMessage
from langchain_ollama import OllamaEmbeddings
from langchain_community.document_loaders import UnstructuredMarkdownLoader
local_llm = "llama3.2:3b-instruct-fp16"
class RAG_by_pdf:
    """
    Define the class that uses PDF as input to implement RAG.
    """

    def __init__(self, question: str, pdf_folder: str, model: str= local_llm,num_retrived_docs: int = 2):
        """
        Initialize the RAG_by_pdf class.

        Args:
        - question (str): The question to be answered.
        - pdf_folder (str): The folder containing the PDF files.
        - num_retrived_docs (int, optional): The number of documents to retrieve. Defaults to 4.
        """
        self.pdf_path = glob.glob(pdf_folder + "/*.pdf")
        self.num_retrived_docs = num_retrived_docs
        self.question = question
        self.model=model
        self.rag_prompt = """You are an assistant for question-answering tasks. 
        Here is the context to use to answer the question:
        {context} 
        Think carefully about the above context. 
        Now, review the user question:
        {question}
        Provide an answer to this questions using only the above context. 
        Answer:"""

    def retriever(self) -> SKLearnVectorStore:
        """
        Create a retriever based on the PDF documents.

        Returns:
        - SKLearnVectorStore: The retriever.
        """
        try:
            # Load documents
            docs = [PyPDFLoader(pdf).load() for pdf in self.pdf_path]
            docs_list = [item for sublist in docs for item in sublist]

            # Split documents
            text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
                chunk_size=1000, chunk_overlap=200
            )
            doc_splits = text_splitter.split_documents(docs_list)

            # Add to vectorDB
            vectorstore = SKLearnVectorStore.from_documents(
                documents=doc_splits,
                #embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
                embedding=OllamaEmbeddings(model=self.model)
            )

            return vectorstore
        except Exception as e:
            print(f"Error creating retriever: {e}")
            return None

    def format_docs(self, docs):
        """
        Format the retrieved documents.

        Args:
        - docs: The retrieved documents.

        Returns:
        - str: The formatted documents.
        """
        return "\n\n".join(doc.page_content for doc in docs)

    def return_context(self) -> str:
        """
        Return the context based on the question.

        Returns:
        - str: The context.
        """
        vectorstore = self.retriever()
        if vectorstore is not None:
            # Create retriever
            retriever = vectorstore.as_retriever(k=self.num_retrived_docs)
            docs = retriever.invoke(self.question)
            docs_txt = self.format_docs(docs)
            rag_prompt_formatted = self.rag_prompt.format(context=docs_txt, question=self.question)
            return rag_prompt_formatted
        else:
            return ""



local_llm = "llama3.2:3b-instruct-fp16"

class RAG_by_markdown:
    """
    Define the class that uses Markdown as input to implement RAG.
    """

    def __init__(self, question: str, markdown_folder: str, model: str= local_llm, num_retrived_docs: int = 2):
        """
        Initialize the RAG_by_markdown class.

        Args:
        - question (str): The question to be answered.
        - markdown_folder (str): The folder containing the Markdown files.
        - model(str): the model used for retriever
        - num_retrived_docs (int, optional): The number of documents to retrieve. Defaults to 4.
        """
        self.markdown_path = glob.glob(markdown_folder + "/*.md")
        self.num_retrived_docs = num_retrived_docs
        self.model=model
        self.question = question
        self.rag_prompt =  """You are an assistant for question-answering tasks. 

        Here is the context to use to answer the question:

        {context} 

        Think carefully about the above context. 

        Now, review the user question:

        {question}

        Provide an answer to this questions using only the above context. 

        Use three sentences maximum and keep the answer concise.

        Answer:"""


    def retriever(self) -> SKLearnVectorStore:
        """
        Create a retriever based on the Markdown documents.

        Returns:
        - SKLearnVectorStore: The retriever.
        """
        try:
            # Load documents
            docs = [UnstructuredMarkdownLoader(md).load() for md in self.markdown_path]
            docs_list = [item for sublist in docs for item in sublist]

            # Split documents
            text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
                chunk_size=1000, chunk_overlap=200
            )
            doc_splits = text_splitter.split_documents(docs_list)

            # Add to vectorDB
            vectorstore = SKLearnVectorStore.from_documents(
                documents=doc_splits,
                #embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
                embedding=OllamaEmbeddings(model=self.model)
            )

            return vectorstore
        except Exception as e:
            print(f"Error creating retriever: {e}")
            return None

    def format_docs(self, docs):
        """
        Format the retrieved documents.

        Args:
        - docs: The retrieved documents.

        Returns:
        - str: The formatted documents.
        """
        return "\n\n".join(doc.page_content for doc in docs)

    def return_context(self) -> str:
        """
        Return the context based on the question.

        Returns:
        - str: The context.
        """
        vectorstore = self.retriever()
        if vectorstore is not None:
            # Create retriever
            retriever = vectorstore.as_retriever(k=self.num_retrived_docs)
            docs = retriever.invoke(self.question)
            docs_txt = self.format_docs(docs)
            rag_prompt_formatted = self.rag_prompt.format(context=docs_txt, question=self.question)
            return rag_prompt_formatted
        else:
            return ""


router_instructions = """You are an expert at routing a user question to a vectorstore.

The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.

Use the vectorstore for questions on these topics.

Return JSON with single key, datasource depending on the question."""

# Define the function to get the output
def get_output(modelname:str,input_type:str,question:str):
    if input_type == "pdf":
        rag = RAG_by_pdf(question, pdf_folder,modelname)
        content = rag.return_context()
        #llm = ChatOllama(model=modelname, temperature=0,num_ctx=4096)
        #generation = llm.invoke([SystemMessage(content=router_instructions)]+[HumanMessage(content=content)])
        #return generation.content
    elif input_type == "markdown":
        rag = RAG_by_markdown(question, md_folder,modelname)
        content = rag.return_context()
    llm = ChatOllama(model=modelname, temperature=0,num_ctx=4096)
    generation = llm.invoke([SystemMessage(content=router_instructions)]+[HumanMessage(content=content)])
    return generation.content

# Create the Gradio interface
demo = gr.Interface(
    fn=get_output,
    inputs=[
        gr.Radio(
            choices=["llama3.2:3b-instruct-fp16", "qwen2.5-coder:14b", "llama3.2-vision:latest"],
            type="value",
            value="llama3.2:3b-instruct-fp16",  # Set default value to "Model 1"
            label="Select Model"
        ),
        gr.Radio(
            choices=["markdown","pdf"],
            type="value",
            value="markdown",  # Set default value to "markdown"
            label="Select Input Type"
        ),
        gr.Textbox(label="Input Questions",info="input questions on chalcogenide perovskites")
    ],
    outputs="text",
    title="RAG Llama3.2-3b based on chalcogenide perovskite papers",
    description="""
    ## Select a model, input file type, and ask a question to get the output; or just click on the examples below.
    only llama3.2-3b with markdown input is tested. may return error in other cases
    """,
    examples=[["llama3.2:3b-instruct-fp16","markdown","what is advantage of BaZrS3?"],
              ["qwen2.5-coder:14b","pdf","what is bandgap of SrHfS3?"],
              ["llama3.2-vision:latest","markdown","why is chalcogenide perovskite importment?"]
              ]
)

# Launch the Gradio app
if __name__ == "__main__":
    demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://67bf2bb1c4b31c3865.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [37]:
import ollama
ollama.list()

ListResponse(models=[Model(model='llama3.2:3b-instruct-fp16', modified_at=datetime.datetime(2024, 12, 12, 17, 13, 29, 530665, tzinfo=TzInfo(-05:00)), digest='195a8c01d91ec3cb1e0aad4624a51f2602c51fa7d96110f8ab5a20c84081804d', size=6433703586, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='3.2B', quantization_level='F16')), Model(model='qwen2.5-coder:14b', modified_at=datetime.datetime(2024, 11, 18, 18, 41, 51, 756602, tzinfo=TzInfo(-05:00)), digest='3028237cc8c52fea4e77185d72cc997b2e90392791f7c82fe1c71995d56e642d', size=8988124256, details=ModelDetails(parent_model='', format='gguf', family='qwen2', families=['qwen2'], parameter_size='14.8B', quantization_level='Q4_K_M')), Model(model='llama3.2-vision:latest', modified_at=datetime.datetime(2024, 11, 18, 17, 41, 39, 984314, tzinfo=TzInfo(-05:00)), digest='38107a0cd11910a31c300fcfd1e9a107b2928e56ebabd14598702170b004773e', size=7901829417, details=ModelDetails(parent_model='', forma

# optimize for kaggle
may need this to load model in model folder to save time.
https://www.gpu-mart.com/blog/import-models-from-huggingface-to-ollama